In [ ]:
import sys
sys.path.append("..")

import random
import math
import itertools
from io import BytesIO
from pathlib import Path
from typing import Optional, Callable, List, Tuple, Iterable, Generator, Union

import PIL.Image
import PIL.ImageDraw
import plotly
import plotly.express as px
plotly.io.templates.default = "plotly_dark"
import numpy as np
import pandas as pd
import clip

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, IterableDataset, RandomSampler
import torchvision.transforms as VT
import torchvision.transforms.functional as VF
from torchvision.utils import make_grid
from IPython.display import display

from src.datasets import *
from src.util.image import *
from src.util import *
from src.algo import *
from src.models.cnn import *

In [ ]:
model, preproc = clip.load("ViT-B/32")

In [ ]:
def plot_weights(images, max_single_shape=(1024, 1024)):
    images = images.copy()
    max_shape = None
    max_single_shape = list(max_single_shape)
    for image_idx, image in enumerate(images):

        if any(a > b for a, b in zip(image.shape, max_single_shape)):
            image = VF.crop(image, 0, 0, min(image.shape[-2], max_single_shape[-2]), min(image.shape[-1], max_single_shape[-1]))
            images[image_idx] = image

        if max_shape is None:
            max_shape = list(image.shape)
        else:
            for i in range(len(max_shape)):
                max_shape[i] = max(max_shape[i], image.shape[i])

    for image_idx, image in enumerate(images):

        if any(a < b for a, b in zip(image.shape, max_shape)):
            images[image_idx] = VF.pad(image, [0, 0, max_shape[-1] - image.shape[-1], max_shape[0] - image.shape[0]])

        images[image_idx] = signed_to_image(images[image_idx])    

    display(VF.to_pil_image(make_grid(images, nrow=10)))
    
images = []
weight = model.visual.conv1.weight
#print(weight.shape)
#for w in weight[0, :10]:
#    images.append(w)
#for w in weight[:10]:
#    images.append(w[0])

for resblock in model.visual.transformer.resblocks[:20]:
    #images.append(resblock.attn.out_proj.weight)
    #images.append(resblock.mlp.c_fc.weight)
    #images.append(resblock.mlp.c_proj.weight.T)
    weight = resblock.attn.out_proj.weight#[:729, :729]
    #print(weight.shape)
    #images.append(weight)
    for w in weight[:10]:
        images.append(w.reshape(32, 24))
    #for w in weight[:10]:
    #    images.append(w.reshape(32, 32))

plot_weights(images)

In [ ]:
768/32

In [ ]:
model